<a href="https://colab.research.google.com/github/Serzh147/Python_based/blob/main/Project_2_Bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Описание проекта
###   Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

импортируем библиотек pandas и открываю файл со статистикой

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data.csv') # прочитаем файл
df.info() # и рассмотрим информацию о нем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df.head() # посмотрим как выглядит таблица

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Чтож, начнем...

# 1. Предобработка данных.

##  1.1  Пропущенные значения.

In [ ]:
df.isna().sum() # посчитаем количесвто пропущенных значений по всей таблице

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Два столбца имеют одинаковое количество пропущенных значений. Ппредполагаю, что между этими столбцами есть связь. Визуализирую датасет, отфильтровав по пропускам в одном из "проблемных" столбцов.

In [ ]:
df[df['days_employed'].isna()].head()# индексация по пропущенным значениям столбца со стажем

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Действительно, просмотр начала и хвоста отфильтрованной по пропускам таблицы показывает совпадение пропущенных значений в обоих столбцах. Очевидно, что между пропусками есть взаимосвязь, и пропуск в столбце дохода связан с наличием такового в столбце со стажем. Или наоборот. Но, так же существует вероятность что эти два количества просто совпали случайным образом, и в середине отфильтрованной таблицы есть несовпадающие пропуски. Проверю. Создам из общего датасета новые таблицы с фильтром по пропущенным значениям в каждом из столбцов стажа и дохода.

In [ ]:
df_employd = df[df['days_employed'].isna()] # Посмотрим инфо по отфильтрованной таблице с пропущенными значениями стажа
df_employd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


In [ ]:
df_income = df[df['total_income'].isna()] # и тоже самое сделаем с фильтром по доходу
df_income.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


В обеих таблицах вижу 0 напротив ненулевых значений в столбце-оппонненте, а это подтвержает, что пропуски в обоих столбцах совпдают полностью. Теперь рассчитаем долю пропущенных значений ко всем представленным данным. Она совпадет для обоих столбцов.

In [ ]:
share = 2174/21525
print(f'Пропущенных значений:{share: .1%}')

Пропущенных значений: 10.1%


Считаю, что 10% пропущенных значений это довольна значимая часть, чтобы повлиять на общий результат исследований. Вижу три способа избавиться от них. Во-первых, можно просто удалить все соответствующие строки. Тогда результат исследований будет точнее, тк основан на более качественных данных, но сузится база выборки ровно на эти 10%. Вряд ли это в интересах заказчика исследования. Во-вторых, можно обнулить пропуски. В таком случае мы сохраним для анализа ценную информацию по этим 10% клиентов в других столбцах, но исказим результат в исследованиях, использующих в рассчетах доход или стаж. Поэтому я склоняюсь к третьему, компромиссному варианту. Рассчитаю медиану для пропущенных значений. Что с одной стороны сохранит смежную используемую информацию по всем клиентам, и при этом не значительно повлияет на расчёты связанные с доходом. Почему медиана, а не иная метрика, например арифметическое среднее? Именно медианна наиболее качественно и верно отражает самые типичные значения, когда речь идет о количественных категориях. Но для примемнения этого инструмента, присутсвующие данные в соответсвующих столбцах сперва нужно проанализировать на наличие аномалий и очистить от них.

####  Вывод по пропускам: похоже произошел како-то технический сбой в сборе или хранении или  выгрузки информации. Ведь если клиентам без стажа могли выдать кредит, то таковым  без дохода врядли. А значит данные дохода просто слетели и "забрали" с собой столбец со стажем. Или наоборот.

##  1.2 Аномалии в данных.

Начнем с "проблемных" Series. Посмотрим из чего состоит стобец days_employed при помощи статистического метода describe

In [ ]:
df['days_employed'].describe() # посмотрим основные статистические метрики, чтобы выявить аномалии

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Отрицательные значения для трудового стажа это очевидная ошибка, исправим ее. 

In [ ]:
df['days_employed'] = df['days_employed'].abs() # применим модуль

In [ ]:
df['days_employed'].describe() # так лучше, но числа все равно аномальные

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

183 года стажа и это только в медиане! А в максиуме и воовсе больше тысячи лет. Очевидно что с этим столбцом что-то не так. Но в задачах исследования нет вопросов, ответ на который зависит от этого параметра. Поэтому оставим пока в таком виде.

In [ ]:
df['total_income'].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

Тут по крайней мере нету отрицательных значений, но для дальнейшей обработки и анализа в этих столбцах пора убрать пропуски. Вместо них так же подставим медианное значение, наиболее типовое для количественных переменных и приведем значения к целочисленным.

In [ ]:
df['total_income'] = df['total_income'].fillna(df['total_income'].median()).astype(int) # сперва для дохода
df['total_income'].head(10)

0    253875
1    112080
2    145885
3    267628
4    158616
5    255763
6    240525
7    135823
8     95856
9    144425
Name: total_income, dtype: int32

In [ ]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median()).astype(int) # сделаем тоже самое для стажа
df['days_employed'].tail(10)

21515       467
21516       914
21517       404
21518    373995
21519      2351
21520      4529
21521    343937
21522      2113
21523      3112
21524      1984
Name: days_employed, dtype: int32

Теперь о детях. Количество вариантов не может быть большим, поэтому применим метод unique и посмотрим с чем имеем дело

In [ ]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [ ]:
df['children'].value_counts() # отсортируем уникальные значения по частоте использования 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

-1 ребенок, будем считать что речь об 1 ребенке. Применим модуль.

In [ ]:
df['children'] = df['children'].abs()

Теперь посмотрим каким клиентам удалось вырастить по 20 детей. Есть подозрение что ноль лишний и речь о двух, но это нужно проверить. Возраст клиентов, по-моему, самый показательный параметр для проверки данного предположения.

In [ ]:
df[df['children'] == 20].groupby('dob_years')['dob_years'].count() # группируем по возрасту "обладателей" 20 детей.

dob_years
0     1
21    1
23    1
24    1
25    1
26    1
27    2
29    2
30    3
31    2
32    2
33    2
34    3
35    2
36    2
37    4
38    1
39    1
40    4
41    2
42    3
43    2
44    2
45    3
46    3
48    1
49    3
50    3
51    1
52    1
53    1
54    1
55    1
56    5
57    1
59    2
60    1
61    1
62    1
64    1
69    1
Name: dob_years, dtype: int64

Действительно, "многодетные" клиенты подозрительно равномерно распределены по возрастной шкале. Похоже что речь всетаки о двух. Производим замену методом replace. 

In [ ]:
df['children'] = df['children'].replace(20, 2)

In [ ]:
df['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [ ]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Вот теперь похоже что информация о детях соответсвует действительности.

In [ ]:
df.info() # проверим, все ли команды сработали

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


Да. В количественных категориях пропуски и аномалии убрали, к более удобному целочисленному типу данные привели. 

##  1.3 Дубликаты.

Начну с выявлеения неявных дубликатов, с целью перевести их в категорию явных и затем удалить все разом. Отсортируем уникальные значения в категориальных Series по частоте использования.

In [ ]:
df['family_status'].value_counts() # тут все нормально

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [ ]:
df['gender'].value_counts() # не думаю что 1 значение XNA  повлияет на результаты, оставляю как есть

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [ ]:
df['income_type'].value_counts() # тут тоже все нормально

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [ ]:
df['education'].value_counts() # а вот тут проблема, значения необходимо привести к единообразному виду

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [ ]:
df['education'] = df['education'].str.lower() # Приведу значения к универсальному виду при помощи метода str.lower()

In [ ]:
df['education'].value_counts() # так лучше

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [ ]:
df.duplicated().sum() # 71 явных дубликатов, от которых нужно избавиться

71

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)  # и удалим их

In [ ]:
df.info() # проверим что получилось, и правда, строк стало меньше ровно на 71 штуку

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int32 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int32 
 11  purpose           21454 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


###  Вывод по 

##  1.4 Категоризация.

Чтобы оптимизировать дальнейшие рассчеты и саму мастер-таблицу и привести ее в удобный читаемый вид, удалим из нее категориальные текстовые Series. Оставив при этом соответсующие им ID и создав из текстовых значений отделльные файлы-словари, к которым можно будет обращаться при необходимости.  

In [ ]:
df_education = df[['education_id','education']] # создание словаря с уровнем образования
df_education = df_education.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов
df_education # словарь готов!

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [ ]:
df_family_status = df[['family_status_id','family_status']] # создание словаря с семейным положением
df_family_status = df_family_status.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов
df_family_status # словарь готов!

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Доходы (столбец total_income) c огромным количеством уникальных значений так же нужно категоризировать  Для этого создадим новый Series 'total_income_category' в котором обозначим к какой группе из 5-ти уровней дохода принадлежит конкретный клиент. А сделаем это через отдельную функцию.

In [ ]:
def total_income_category(income):
    """
    Возвращает группу дохода клиента по значению уровня дохода income, используя правила:
    - 'E', если income <= 30000 ;
    - 'D', если income 30001–50000 ;
    - 'C', если income 50001–200000 ;
    - 'B', если income 200001–1000000 ;
    - 'A', если income >= 1000001 ; 
    """
    if income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A' 

Создадим новый столбец 'total_income_category', в который применим эту функцию для доходов(столбец total_income) с помощью метода apply

In [ ]:
df['total_income_category'] = df['total_income'].apply(total_income_category)
df.head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C


Получилось, но не очень красиво, что новый столбец попал в самый конец таблицы. Переместим его в место, рядом с исходным столбцом с доходами. Да, я планирую столбец с доходами в будущем совсем удалить, но не хочется нарушать существующую логику колонок. Для этого методом columns вызову названия колонок, и при помощи индексации создам новый df с желаемым расположением.

In [ ]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'total_income_category'],
      dtype='object')

In [ ]:
df = df[['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income','total_income_category','purpose']]
df.head() # внутрненний перфекционист выдохнул

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,total_income_category,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,B,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,C,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,C,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,B,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,C,сыграть свадьбу


In [ ]:
df['purpose'].value_counts() # Посмотрим из каких значений состоит колонка с целями кредитования

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Очень много разных целей, суть которых можно свести к 4-м категориям:
 - операции с автомобилем,
 - операции с недвижимостью,
 - проведение свадьбы,
 - получение образования.
 
Реализуем это так же через создание функции.

In [ ]:
list_df = df['purpose'].tolist() # Преобразуем Series в список
def purpose_category(list_df): # объявим функцию, которая пройдет циклом по всему списку и вернет категории
    """  Возвращает обобщенную категорию целей кредита по ключевым словам """
    for cell in list_df:
        if 'авто' in list_df:
            return 'операции с автомобилем'
        elif 'недвиж' in list_df:
            return 'операции с недвижимостью'
        elif 'жиль' in list_df:
            return 'операции с недвижимостью'
        elif 'свадь' in list_df:
            return 'проведение свадьбы'
        elif 'образован' in list_df:
            return 'получение образования'
        else:
            return 'None' # оставим для проверки, все ли значения удалось обработать и заменить

 

In [ ]:
df['purpose_category'] = df['purpose'].apply(purpose_category)

In [ ]:
df['purpose_category'].value_counts() # Да, похоже функция сработала как нужно

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

#   Анализ

Для начала создам таблицу без лишних столбцов. Все дальнейшие рассчеты уже буду делать по ней, подтягивая названия категорий из словарей функцией объеденения таблиц "merge"

In [ ]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'total_income_category', 'purpose', 'purpose_category'],
      dtype='object')

In [ ]:
df_cut = df[['children', 'education_id', 'family_status_id','gender','debt', 'total_income_category','purpose_category']] # создание словаря с уровнем образования
df_cut.head(10) # 
df_cut.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21454 non-null  int64 
 1   education_id           21454 non-null  int64 
 2   family_status_id       21454 non-null  int64 
 3   gender                 21454 non-null  object
 4   debt                   21454 non-null  int64 
 5   total_income_category  21454 non-null  object
 6   purpose_category       21454 non-null  object
dtypes: int64(4), object(3)
memory usage: 1.1+ MB


####   Таблица готова. Далее будем дейстовать по следующему шаблону. Создадим сводную таблицу в которой данные будут группироваться по интересующей нас категории и подсичтываться процент проблемных кредитов по отношению к хорошим.

## Дети.

In [ ]:
df_cut_children = df_cut.pivot_table(index=['children'], columns='debt', values = 'gender', aggfunc='count')
df_cut_children['ratio'] = df_cut_children[1]/df_cut_children[0]*100
df_cut_children

debt,0,1,ratio
children,,,
0,13028.0,1063.0,8.159349
1,4410.0,445.0,10.090703
2,1926.0,202.0,10.488058
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,NaN,NaN


8% бездетных семей имели проблемы с возвратом кредита, а среди семей с детьми этот параметр 10% и более. Исключение составляет лишь семьи с 3мя детьми, но это вдимо какой-то выброс или особенность выборки, т.к. их доля не велика (около 1.5%), а тенденция к увеличению невозвратов с увеличеснием количества детей прослеживается четко. Еще остался вопрос по семьям с 5-тью детьми. Почему Nan? Осортируем по количесвту детей и посмотрим что с возвратом у них.

In [ ]:
df_cut[['children','debt']].sort_values(by='children').tail(10)

,children,debt
8889,4,0
3977,5,0
7859,5,0
20770,5,0
15787,5,0
15881,5,0
16173,5,0
4394,5,0
20387,5,0
21087,5,0


Оказывается у них не было проблем с возвратами впринципе, но количсетво таких семей совсем мизерное.

## Семейное положение

In [ ]:
df_cut_family_status = df_cut.pivot_table(index=['family_status_id'], columns='debt', values = 'gender', aggfunc='count')
df_cut_family_status['ratio'] = df_cut_children[1]/df_cut_children[0]*100
df_cut_family_status

debt,0,1,ratio
family_status_id,,,
0,11408,931,8.159349
1,3763,388,10.090703
2,896,63,10.488058
3,1110,85,8.910891
4,2536,274,10.810811


In [ ]:
df_cut_family_status = df_cut_family_status.merge(df_family_status, on='family_status_id', how='left')
df_cut_family_status.sort_values(by='ratio')

,family_status_id,0,1,ratio,family_status
0,0,11408,931,8.159349,женат / замужем
3,3,1110,85,8.910891,в разводе
1,1,3763,388,10.090703,гражданский брак
2,2,896,63,10.488058,вдовец / вдова
4,4,2536,274,10.810811,Не женат / не замужем


Самый низкий процент "плохих" кредитов у жеантых пар. Одинокие люди показывают меньшую финансовую ответсвенность.

## Доход

In [ ]:
df_total_income = df_cut.pivot_table(index=['total_income_category'], columns='debt', values = 'gender', aggfunc='count')
df_total_income['ratio'] = df_total_income[1]/df_total_income[0]*100
df_total_income.sort_values(by='ratio')

debt,0,1,ratio
total_income_category,,,
D,329,21,6.382979
B,4685,356,7.598719
A,23,2,8.695652
C,14656,1360,9.279476
E,20,2,10.000000


Очень интересно разбросало катергории, оказывается самые ответственные заемщики это не самые состоятельные люди. Люди с наименьшим доходом ожидаемо в группе риска.

## Цели

In [ ]:
df_purpose = df_cut.pivot_table(index=['purpose_category'], columns='debt', values = 'gender', aggfunc='count')
df_purpose['ratio'] = df_purpose[1]/df_purpose[0]*100
df_purpose.sort_values(by='ratio')

debt,0,1,ratio
purpose_category,,,
операции с недвижимостью,10029,782,7.797388
проведение свадьбы,2138,186,8.699719
получение образования,3643,370,10.156464
операции с автомобилем,3903,403,10.325391


Оказалось что для банка самыми безопасными являются кредиты выданные на операции с недвижимостью. Автокредитование и образование в зоне риска.

## Общий вывод

Результаты исследования показали, что по ранее выданным кредитам ситуация следующая. Самый ответсвенный заемщик для банка - это бездетная семейная пара со средним уровнем дохода, берущая кредит на операции с недвижимостью. Собирательный образ проблемного заемщика можно охарактеризовать как одинокие люди с детьми, имеющие доход значительно выше или так же значительно ниже среднего, берущие автокредит.